In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from aegomoku.constants_19x19 import *
from aegomoku.gomoku_game import Swap2
from aegomoku.gomoku_players import PolicyAdvisedGraphSearchPlayer
from aegomoku.interfaces import MctsParams
from aegomoku.policies.topological_value import TopologicalValuePolicy
from aegomoku.interfaces import PolicyParams, PolicyAdviser
from aegomoku.interfaces import Game, Player
import copy

In [5]:
BOARD_SIZE=19

In [6]:

game = Swap2(BOARD_SIZE)


policy_params = PolicyParams(model_file_name=None, advice_cutoff=.01)
policy = TopologicalValuePolicy(kappa_s=6, kappa_d=5)
adviser = PolicyAdviser(model=policy, params=policy_params)

mcts_params = MctsParams(cpuct=4.0, temperature=3.0, num_simulations=10)
player1 = PolicyAdvisedGraphSearchPlayer("Swappy Joe", game, mcts_params)
player2 = PolicyAdvisedGraphSearchPlayer("Sleepy Jack", game, mcts_params)

board = GomokuBoard(BOARD_SIZE)

In [5]:
from aegomoku.gomoku_game import one_game
one_game(game, player1, player2, max_moves=20)

O1 (White next)


All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.


Oops!
Got None of type <class 'NoneType'>


ValueError: If a single argument is provided, it must be a string or integer representation of the move.

In [6]:
board = GomokuBoard(BOARD_SIZE)

In [16]:
policy = TopologicalValuePolicy(kappa_s=6, kappa_d=5)
board = GomokuBoard(BOARD_SIZE, stones=[200])
center_value = policy.evaluate(board.canonical_representation())[1] / 2
center_value

-0.04081839695572853

In [24]:
from tqdm import tqdm
import random

d = 5
corner = [i*BOARD_SIZE + j for i in range(d) for j in range(d)]
left = [(i + d) *BOARD_SIZE + j for i in range(d) for j in range(d)]
top = [i * BOARD_SIZE + j + d for i in range(d) for j in range(d)]
perimeter = corner + left + top

candidates = []
dist = 1
for i in tqdm(range(100)):
    while True:
        moves = [board.Stone(random.choice(perimeter)) for _ in range(3)]
        try:
            board = GomokuBoard(BOARD_SIZE, stones=moves)
            value = policy.evaluate(board.canonical_representation())[1]
            dist = (value - center_value) ** 2
            break
        except AssertionError as e:
            continue
    candidates.append((moves, dist))


most_neutral = sorted(candidates, key=lambda e: e[1])[0][0]
most_neutral

100%|██████████| 100/100 [00:00<00:00, 222.67it/s]


[J18, E12, B13]

In [25]:
r = range(d, BOARD_SIZE-d+1)
center = [i*BOARD_SIZE + j for i in r for j in r]

candidates = []
for i in tqdm(range(100)):
    while True:
        moves = [board.Stone(random.choice(center)) for _ in range(2)]
        try:
            moves = most_neutral + moves
            board = GomokuBoard(BOARD_SIZE, stones=moves)
            value = policy.evaluate(board.canonical_representation())[1]
            dist = (value - center_value) ** 2
            break
        except AssertionError as e:
            continue
    candidates.append((moves, dist))

most_neutral = sorted(candidates, key=lambda e: e[1])[:5]


100%|██████████| 100/100 [00:00<00:00, 221.57it/s]


In [26]:
choice = sorted(most_neutral, key=lambda e: e[1])[:5]
for stones, _ in choice:
    GomokuBoard(BOARD_SIZE, stones=stones).plot()

                                                                 
19    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
18    .  .  .  .  .  .  .  .  .  X  .  .  .  .  .  .  .  .  .    
17    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
16    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  X  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  O  .  .  .  .  . [X] .  .  .  .  .  .  .  .    
11    .  .  .  .  .  .  .  .  .  .  O  .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 9    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 8    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 7    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 5    .  .

In [27]:
board = GomokuBoard(BOARD_SIZE, stones=[200])
center = policy.evaluate(board.canonical_representation())[1] / 2
center

-0.04081839695572853

### All of that is now available with the TopSwap2BoardInitializer

In [7]:
from aegomoku.gomoku_game import TopoSwap2BoardInitializer

sbi = TopoSwap2BoardInitializer(19)
GomokuBoard(BOARD_SIZE, stones=sbi.initial_stones()).plot()

                                                                 
19    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
18    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
17    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
16    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
11    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 9    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 8    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 7    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 5    .  .

In [9]:
sbi.initial_stones()

'A19E19E18I14K9'